In [1]:
import sys
sys.path.append('..')
from lib import *
import pandas as pd
import numpy as np

forest = RNF(None, None, None, None, None, None, None)
forest.load_rnf('scen1.first600.64trees.10depth.42rs.20maxfeats.600maxin.pickle')

In [2]:
d = forest.get_feature_importances()

In [3]:
sorted([(d[key], key) for key in d.keys()], reverse=True)

[(0.30477569800839904, '86'),
 (0.25722202191478694, '40'),
 (0.2090195068346005, '74'),
 (0.19609119218159657, '45'),
 (0.17069576734674177, '82'),
 (0.15095820798827475, '0'),
 (0.1402478928662062, '72'),
 (0.13725012651064744, '3'),
 (0.13161104561248824, '26'),
 (0.13079356072698495, '19'),
 (0.13010217882116437, '59'),
 (0.1290920631441454, '1'),
 (0.119556598605028, '50'),
 (0.09473832223010924, '34'),
 (0.0822009503841754, '23'),
 (0.08037666981570793, '93'),
 (0.0776098481310565, '36'),
 (0.07715352183461222, '20'),
 (0.07227730793380041, '66'),
 (0.07080871956318346, 'Date'),
 (0.06481262537098637, '49'),
 (0.06454402748906067, '60'),
 (0.06262095227320935, '44'),
 (0.061246786316304806, '33'),
 (0.058973720232189705, '17'),
 (0.05798506181352521, '2'),
 (0.057740170049957816, '47'),
 (0.05753142764173777, '80'),
 (0.05650618258577356, '31'),
 (0.05608563289214151, '32'),
 (0.0539455719104616, '11'),
 (0.05104286733995867, '22'),
 (0.05079745740135234, '85'),
 (0.0499889360418